In [1]:
from SOM import *
from PZC import *
from TomographicBins import *
from pipeline_tools import *

from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [2]:
pz = load_PZC('../outputs/basic/')

wide_SOM = pz.wide_SOM
deep_SOM = pz.deep_SOM

## Catalog Checking

### Wide Sample

In [ ]:
wide_som_sample = wide_SOM.validate_sample ; bands = ['g','r','i','z']
# range=(-1000000,-10), 
[plt.hist(wide_som_sample['Mf_%s'%b], 300, histtype='step', label=b) for b in bands]
plt.legend()
plt.yscale('log') ; plt.xscale('symlog')
#plt.xlim(-1000000,0)
plt.show()

### Deep Sample

In [ ]:
deep_som_sample = deep_SOM.validate_sample ; bands = list('ugrizjhk')
# 
[plt.hist(deep_som_sample['Mf_%s'%b], 300, range=(0, 1000), histtype='step', label=b) for b in bands]
plt.legend()
#plt.yscale('log') #; plt.xscale('symlog')
#plt.xlim(-1000000,0)
plt.show()

In [ ]:
plt.hist(deep_som_sample['COSMOS_PHOTZ'], 300, histtype='step')
print(wide_som_sample.colnames)
plt.hist(wide_som_sample['DNF_Z_MC'], 300, histtype='step')
plt.show()

# SOM Validation

## Wide SOM

In [ ]:
occ = wide_SOM.get('occupation')

plt.imshow(occ)
plt.colorbar()
plt.title("Wide Occupation")

plt.show()

In [ ]:
bands = ['g','r','i','z']
means = [wide_SOM.get('mean','Mf_%s'%b) for b in bands]
stds = [wide_SOM.get('std','Mf_%s'%b) for b in bands]

means_mag = [flux_to_mag(m) for m in means]
stds_mag = [flux_to_mag(s) for s in stds]

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(20,10))
for i, (b1, b2) in enumerate(zip(bands, bands[1:])):
    im1=ax[0,i].imshow(means_mag[i]-means_mag[i+1])
    ax[1,i].imshow(stds_mag[i]-stds_mag[i+1])
    
    divider = make_axes_locatable(ax[0,i])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im1, cax=cax, orientation='vertical')
    
    ax[0,i].set_title('%s-%s'%(b1,b2))
    
plt.show()

## Deep SOM

In [ ]:
occ = deep_SOM.get('occupation')

plt.imshow(occ)
plt.colorbar()
plt.title("Deep Occupation")

plt.show()

In [ ]:
bands = ['u','g','r','i','z','j','h','k']
means = [deep_SOM.get('mean','Mf_%s'%b) for b in bands]
stds = [deep_SOM.get('std','Mf_%s'%b) for b in bands]

means_mag = [flux_to_mag(m) for m in means]
stds_mag = [flux_to_mag(s) for s in stds]

In [ ]:
fig, ax = plt.subplots(2,len(bands)-1, figsize=(20,10))
for i, (b1, b2) in enumerate(zip(bands, bands[1:])):
    im1=ax[0,i].imshow(means_mag[i]-means_mag[i+1])
    ax[1,i].imshow(stds_mag[i]-stds_mag[i+1])
    
    divider = make_axes_locatable(ax[0,i])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im1, cax=cax, orientation='vertical')
    
    ax[0,i].set_title('%s-%s'%(b1,b2))
    
plt.show()

# Tomographic Binning

In [3]:
five_percent = load_result('../outputs/basic/nzs/Nz.pkl')

In [ ]:
compost_std = np.array([D(five_percent.z, five_percent.pzchat[wc]) for wc in five_percent.compost_WCs])
bin_stds = [np.array([D(five_percent.z, pzi) for pzi in five_percent.pzchat[WCs]] )
                     for WCs in five_percent.binned_WCs]

compost_mm = [E(five_percent.z, pzi) for pzi in five_percent.pzchat[five_percent.compost_WCs]]
bin_mms = [[E(five_percent.z, pzi) for pzi in five_percent.pzchat[WCs]] for WCs in five_percent.binned_WCs]

fig, axs = plt.subplots(2, figsize=(20,10))
for a,comp,bins in zip(axs, [compost_std, compost_mm], [bin_stds, bin_mms]):
    a.hist(comp, 30, label='compost', histtype='step')
    [a.hist(b, 30, label="bin %i"%i, histtype='step') for i,b in enumerate(bins)]

axs[0].set_ylabel("Bin Standard \nDeviations", fontsize=20)
axs[1].set_ylabel("Bin Expectation \nValues", fontsize=20)
plt.legend(fontsize=20)
plt.show()

In [ ]:
distributions = five_percent.calculate_Nz()
distributions_with_selef = five_percent.calculate_Nz(apply_selection_effects=True)

In [ ]:
zi = five_percent.z
z = np.arange(0,6,0.01)

colors = [plt.cm.tab20(i*2) for i in range(5)]
fig, ax = plt.subplots(2, figsize=(25,17), height_ratios=[6,1])
for i in distributions:
    if i == -1: continue
    nz_selef = distributions_with_selef[i]
    ax[0].fill_between(z,savgol_filter(nz_selef,18,1), alpha=0.3, color=colors[i], label="Bin %i with Selection Effects"%i)
    nz = distributions[i] ; av = E(zi,nz) 
    ax[0].plot(zi, savgol_filter(nz, 18, 1), color=colors[i], label="Bin %i (avg = %.02f)"%(i,av))

    #ax[0].axvline(av, color=colors[i], label='bin average = %.02f'%av)


nz_selef = distributions_with_selef[0]
ax[1].fill_between(z, savgol_filter(nz_selef,18,1), alpha=0.3, color=colors[-1], label="Compost with Selection Effects")
nz = distributions[-1]
ax[1].plot([0]+zi, [0]+savgol_filter(nz, 18,1), color=colors[-1], label='Compost Bin')
av = E(zi,nz) 
#ax[1].axvline(av, color=colors[-1], label='bin average = %.02f'%av)

[a.set_ylim(bottom=0) for a in ax]
[a.set_xlim(left=-0.05) for a in ax]

ax[0].legend(fontsize=25)
ax[1].legend(fontsize=25)
ax[0].tick_params(axis='both', which='major', labelsize=25)
ax[1].tick_params(axis='both', which='major', labelsize=25)

ax[1].set_xlabel('Redshift', fontsize=25)


In [4]:
assignments, grouped_by_tbin = five_percent.bin_sample(wide_SOM.validate_sample)

100%|██████████| 1000/1000 [04:42<00:00,  3.54it/s]
